In [1]:
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime
from collections import namedtuple
from pyspark.sql import types
import pandas as pd

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/17 16:26:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/17 16:26:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [5]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [11]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [6]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [7]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [8]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [9]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [12]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [14]:
df_result.take(1)

[Row(hour=datetime.datetime(2020, 1, 20, 15, 0), zone=67, revenue=79.5, count=3)]

In [15]:
df_result.write.parquet('tmp/green-revenue')

### RDD mapPartition 

In [16]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [17]:
rows = duration_rdd.take(10)

In [21]:
# turn to pandas DF
df = pd.DataFrame(rows, columns=columns)

In [22]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [23]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [30]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index','lpep_pickup_datetime')

In [31]:
df_predicts.show()

+--------+------------+------------+-------------+------------------+
|VendorID|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+------------+------------+-------------+------------------+
|     2.0|          74|         130|        12.77|63.849999999999994|
|     NaN|          67|          39|          8.0|              40.0|
|     2.0|         260|         157|         1.27|              6.35|
|     2.0|          82|          83|         1.25|              6.25|
|     2.0|         166|          42|         1.84| 9.200000000000001|
|     2.0|         179|         223|         0.76|               3.8|
|     2.0|          41|         237|         3.32|16.599999999999998|
|     2.0|          75|         161|         2.21|             11.05|
|     1.0|         152|         166|          0.9|               4.5|
|     2.0|          75|         234|          6.1|              30.5|
|     2.0|          75|          41|         1.74|               8.7|
|     2.0|         2